In [48]:
'''
@Author: Venkatesh <br>
@Date: 2024-11-19  <br>
@Last Modified by: Venkatesh <br>
@Last Modified time: 2024-11-19 <br>
@Title: predicting whether customer will subscribe to term deposit or not using smulti layer neural network  <br>

'''

'\n@Author: Venkatesh <br>\n@Date: 2024-11-19  <br>\n@Last Modified by: Venkatesh <br>\n@Last Modified time: 2024-11-19 <br>\n@Title: predicting whether customer will subscribe to term deposit or not using smulti layer neural network  <br>\n\n'

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os

Load the data set

In [50]:
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 
    'marital_status', 'occupation', 'relationship', 'race', 'sex', 
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'
]

# Read the CSV file without headers and assign column names
dataset = pd.read_csv("D:\\ML\\neural_network\\Salary_Data.csv", header=None, names=column_names,na_values=[' ?'])

print(dataset.info())

categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']
for column in categorical_columns:
    mode_value = dataset[column].mode()[0]
    dataset[column].fillna(mode_value, inplace=True)

dataset.info()
print(dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns

C:\Users\Venkatesh Bingi\AppData\Local\Temp\ipykernel_7628\2408216417.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(mode_value, inplace=True)


In [51]:
# Display the first few rows of the dataset
print(dataset.head())

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50

In [52]:
# Handle missing data
print("\nDataset Information:")
print(dataset.info())



Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None


In [53]:
# Check for missing values
print("\nMissing values in dataset:\n", dataset.isnull().sum())


Missing values in dataset:
 age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64


Encoding categorical data

In [54]:
# Encode categorical variables
label_encoder = LabelEncoder()
categorical_cols = dataset.select_dtypes(include=['object']).columns

for col in categorical_cols:
    dataset[col] = label_encoder.fit_transform(dataset[col])

print(dataset.head())


   age  workclass  fnlwgt  education  education_num  marital_status  \
0   39          6   77516          9             13               4   
1   50          5   83311          9             13               2   
2   38          3  215646         11              9               0   
3   53          3  234721          1              7               2   
4   28          3  338409          9             13               2   

   occupation  relationship  race  sex  capital_gain  capital_loss  \
0           0             1     4    1          2174             0   
1           3             0     4    1             0             0   
2           5             1     4    1             0             0   
3           5             0     2    1             0             0   
4           9             5     2    0             0             0   

   hours_per_week  native_country  income  
0              40              38       0  
1              13              38       0  
2              40   

Future scalling

In [55]:
# Split dataset into features (X) and target (y)
X = dataset[['workclass', 'education','marital_status']]  # Assuming 'income' is the target column
y = dataset['income']

# Encode target variable
y = label_encoder.fit_transform(y)  # Convert 'income' to 0/1


In [56]:
# Encode the target variable
y = label_encoder.fit_transform(y)  # Convert 'income' to 0/1 (binary classification)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Building model

In [57]:
# Step 5: Build the Artificial Neural Network (ANN) with multiple hidden layers
model = Sequential()

# First hidden layer
model.add(Dense(units=32, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
model.add(Dense(units=16, activation='relu'))

# Third hidden layer
model.add(Dense(units=8, activation='relu'))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))  # Sigmoid for binary classification

# Step 6: Compile the ANN
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 7: Train the ANN
model.fit(X_train_scaled, y_train, epochs=15, batch_size=32, validation_split=0.2)

# Step 8: Predict on the test set
y_pred = model.predict(X_test_scaled)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to 0/1

Epoch 1/15


c:\Users\Venkatesh Bingi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


652/652 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7635 - loss: 0.4820 - val_accuracy: 0.7879 - val_loss: 0.4334
Epoch 2/15
652/652 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7942 - loss: 0.4205 - val_accuracy: 0.8017 - val_loss: 0.4246
Epoch 3/15
652/652 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8038 - loss: 0.4215 - val_accuracy: 0.8140 - val_loss: 0.4210
Epoch 4/15
652/652 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8099 - loss: 0.4102 - val_accuracy: 0.8140 - val_loss: 0.4158
Epoch 5/15
652/652 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8134 - loss: 0.4097 - val_accuracy: 0.8144 - val_loss: 0.4112
Epoch 6/15
652/652 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8100 - loss: 0.4057 - val_accuracy: 0.8155 - val_loss: 0.4098
Epoch 7/15
652/652 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8068 - loss: 0.4120 - val_accuracy: 0.8171 - val_loss: 0.4073
Epoch 8/15
652/652 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8167 - loss: 0.4019 - val_accuracy: 0.8052 - val_

In [58]:
# Step 9: Evaluate the model performance

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)

# Display results
print("Confusion Matrix:\n", cm)
print("Accuracy:", accuracy)

Confusion Matrix:
 [[4569  373]
 [ 844  727]]
Accuracy: 0.8131429448794718
